In [1]:
from pathlib import Path

import pandas as pd
import altair as alt

import ideafast_deviceselection as ifds

source = Path(ifds.__file__).parent.parent.absolute() / 'local/DeviceSelectionScoring.xlsx'


df = pd.read_excel(
    f"{source}",
    index_col=0, header=[0,1], nrows=63,
    sheet_name='MASTER SCORES')
df.drop(df.columns[0:12],axis=1,inplace=True)
df.drop(df.index[0],axis=0,inplace=True)
df.drop(df.columns[30:],axis=1,inplace=True)

# extend empty headers to use the multiIndex
a = df.columns.get_level_values(0)
b = df.columns.get_level_values(1)
df.columns = [a.to_series().mask(lambda x: x.str.startswith('Unnamed')).ffill(), b]

df.index.names = ['Criteria']
df.columns.names = ['device','type']

df.drop('RELEVANCE * SCORE',axis=1,level=1,inplace=True)

df = df.reset_index()

df

device Criteria Vital Patch           VTT EBED           VTT SMA            \
type                  SCORE CERTAINTY    SCORE CERTAINTY   SCORE CERTAINTY   
0            1a    0.850000  0.153846    0.650  0.153846   0.700  0.153846   
1            1b         NaN       NaN      NaN       NaN     NaN       NaN   
2            1c         NaN       NaN      NaN       NaN     NaN       NaN   
3            1d    0.850000  0.106667    0.750  0.093333   0.850  0.106667   
4          1e_1         NaN       NaN      NaN       NaN     NaN       NaN   
..          ...         ...       ...      ...       ...     ...       ...   
57           5i    0.500000  0.230769    0.900  0.230769   0.500  0.230769   
58           5j    0.641667  0.800000    0.750  0.966667   0.875  0.966667   
59           5k    0.562500  0.600000    0.525  1.000000   0.950  0.250000   
60           5l    0.900000  0.666667    0.750  1.000000   0.500  0.666667   
61           5m    0.900000  0.500000    1.000  0.500000   1.000  0.500000   

device Cambridge Cognition             Axivity  ... McRoberts            \
type                 SCORE CERTAINTY     SCORE  ...     SCORE CERTAINTY   
0                    1.000  0.769231  0.900000  ...  0.800000  0.153846   
1                      NaN       NaN       NaN  ...       NaN       NaN   
2                      NaN       NaN       NaN  ...       NaN       NaN   
3                    0.950  0.106667  0.850000  ...  0.850000  0.106667   
4                      NaN       NaN       NaN  ...       NaN       NaN   
..                     ...       ...       ...  ...       ...       ...   
57                   0.500  0.230769  0.900000  ...  0.900000  0.230769   
58                   0.975  0.966667  0.558333  ...  0.550000  0.966667   
59                   0.650  0.250000  0.433333  ...  0.350000  0.750000   
60                   0.500  0.666667  0.500000  ...  0.416667  1.000000   
61                   1.000  0.500000  1.000000  ...  0.900000  0.500000   

device     ZKOne           Biovotion Everion               Dreem            \
type       SCORE CERTAINTY             SCORE CERTAINTY     SCORE CERTAINTY   
0       0.800000  0.153846               NaN       NaN  0.850000  0.384615   
1            NaN       NaN               NaN       NaN       NaN       NaN   
2            NaN       NaN               NaN       NaN       NaN       NaN   
3       0.650000  0.066667              0.85  0.066667  0.850000  0.066667   
4            NaN       NaN               NaN       NaN       NaN       NaN   
..           ...       ...               ...       ...       ...       ...   
57      0.900000  0.230769               NaN       NaN  0.750000  0.230769   
58      0.666667  0.966667              0.00  0.133333  0.833333  0.966667   
59      0.316667  0.650000               NaN       NaN  0.600000  0.500000   
60      0.733333  1.000000               NaN       NaN  1.000000  0.666667   
61      1.000000  0.500000               NaN       NaN  0.900000  0.500000   

device Byteflies            
type       SCORE CERTAINTY  
0            NaN       NaN  
1            NaN       NaN  
2            NaN       NaN  
3            NaN       NaN  
4            NaN       NaN  
..           ...       ...  
57      0.650000  0.230769  
58      0.900000  0.833333  
59      0.466667  0.500000  
60      0.500000  0.666667  
61      0.900000  0.500000  

[62 rows x 21 columns]

In [2]:
categories = ["Data Quality, Reliability & Analytics", "Data Access, Transparency & Handling", "Accessibility, Usability & User Experience", "Regulatory Concerns", "Scalability & Practicality", "Track Record & Data Availability"]

long = pd.melt(df, id_vars=['Criteria'])
long.insert(0,'group',[categories[int(x)] for x in long['Criteria'].str[0]])

long

,group,Criteria,device,type,value
0,"Data Access, Transparency & Handling",1a,Vital Patch,SCORE,0.850000
1,"Data Access, Transparency & Handling",1b,Vital Patch,SCORE,NaN
2,"Data Access, Transparency & Handling",1c,Vital Patch,SCORE,NaN
3,"Data Access, Transparency & Handling",1d,Vital Patch,SCORE,0.850000
4,"Data Access, Transparency & Handling",1e_1,Vital Patch,SCORE,NaN
...,...,...,...,...,...
1235,Track Record & Data Availability,5i,Byteflies,CERTAINTY,0.230769
1236,Track Record & Data Availability,5j,Byteflies,CERTAINTY,0.833333
1237,Track Record & Data Availability,5k,Byteflies,CERTAINTY,0.500000
1238,Track Record & Data Availability,5l,Byteflies,CERTAINTY,0.666667


In [3]:
# grouped = long.groupby(['group','device','type'])
# grouped

In [4]:
scored = long.loc[long['type'] == "SCORE"]

scored

,group,Criteria,device,type,value
0,"Data Access, Transparency & Handling",1a,Vital Patch,SCORE,0.850000
1,"Data Access, Transparency & Handling",1b,Vital Patch,SCORE,NaN
2,"Data Access, Transparency & Handling",1c,Vital Patch,SCORE,NaN
3,"Data Access, Transparency & Handling",1d,Vital Patch,SCORE,0.850000
4,"Data Access, Transparency & Handling",1e_1,Vital Patch,SCORE,NaN
...,...,...,...,...,...
1173,Track Record & Data Availability,5i,Byteflies,SCORE,0.650000
1174,Track Record & Data Availability,5j,Byteflies,SCORE,0.900000
1175,Track Record & Data Availability,5k,Byteflies,SCORE,0.466667
1176,Track Record & Data Availability,5l,Byteflies,SCORE,0.500000


In [11]:
alt.Chart(scored).mark_boxplot().encode(
    x='device',
    y='value',
).facet(
     column='group'
)

alt.FacetChart(...)